In [12]:
awg = AWG_M8190A_qudi
import hardware.awg.multi_channel_awg_seq as mcas

awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
awg.awg._ch[2].sample_marker_low = 0
awg.awg._ch[2].sample_marker_high = 1.75
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.5

In [13]:
awg.awg._ch[1].sync_marker_high=1.5
awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.5



In [14]:

seq = mcas.MultiChSeq('Laser', ch_dict = {'awg':[1,2]})
seq.awg = awg
seq.start_new_segment('Starting segment')


seq.asc(length_mus = 1, green=True)
seq.asc(length_mus = 1)
# seq.asc(pdawg1=dict(type='sine', frequencies=[2100], amplitudes=[0.3]),
#         length_mus=10)
# seq.asc(length_mus = 10, green=False, orange = True) #green on, AOM off
# seq.asc(length_mus = 10, green=False, orange = False)


# seq.sequences['awg'][1].print_info()

In [15]:
seq.write_seq()
seq.run_sequence()

MCAS 'Laser' has been generated.
writing sequence Laser on AWG_M8190A_qudi ch 1 took 0.0110 seconds
writing sequence Laser on AWG_M8190A_qudi ch 2 took 0.0110 seconds


In [11]:
seq.stop_awgs()

writing sequence wait on AWG_M8190A_qudi ch 1 took 0.0100 seconds
writing sequence wait on AWG_M8190A_qudi ch 2 took 0.0090 seconds


In [ ]:
1/7e-10

In [25]:
awg = AWG_M8190A_qudi
import hardware.awg.multi_channel_awg_seq as mcas

awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.5

def awg_readout_test(taus):
    seq = mcas.MultiChSeq('hahn_echo', ch_dict={'awg': [1, 2]})
    phases = [0.,180.]
    awg_freq = 2206.54 # in MHz
    electron_amp = 0.2 # AU 300mV peak to peak
    electron_pi = 1.352/2 # us
    oscReadoutPhaseFreq = 0
    nitrogen_transition = 14.0733
    rf_safety = 1

#     seq.start_new_segment('hahn')
#     seq = awg_AddWait(seq, 3)
#     seq = awg_AddReadout(seq)


#     seq.start_new_segment('hahn')
    
#     seq.asc(length_mus = 3000, green=True, orange = True)

#     seq = awg_AddWait(seq, 3)
#     seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp]),
#             length_mus=electron_pi)
#     seq = awg_AddWait(seq, 3)
#     seq = awg_AddReadout(seq)


    seq.start_new_segment('hahn')
    
    seq.asc(length_mus = 3000, green=True, orange = True)
    seq.asc(length_mus=.5)  # 2nd step
    
#     seq = awg_AddWait(seq, 3)
#     seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp]),
#             length_mus=electron_pi)
#     seq = awg_AddWait(seq, 3)
    seq = awg_AddReadout(seq)
    
    
    return seq


def awg_green_ramsey(taus):
    seq = mcas.MultiChSeq('hahn_echo', ch_dict={'awg': [1, 2]})
    phases = [0.,180.]
    awg_freq = 2206.54 # in MHz
    electron_amp = 0.2 # AU 300mV peak to peak
    electron_pi = 1.352/2 # us
    oscReadoutPhaseFreq = 0
    nitrogen_transition = 14.0733
    rf_safety = 1
    
    for tau in taus:
        seq.start_new_segment('hahn')

        
        seq = awg_AddWait(seq, tau)

        seq.asc(length_mus=0.1)
        
        seq = awg_AddReadout(seq)

    
    return seq



def awg_AddReadout(seq):
    length_laserreadout_ns = 3000
    length_lasersafety_ns = 4000
    AOMdelay_ns = 400
    length_laserreadout_smpl = length_laserreadout_ns / 1e3 * 12e3
    AOMdelay_sample = AOMdelay_ns  / 1e3 * 12e3
    
    while length_laserreadout_smpl % 64 != 0:
        length_laserreadout_smpl += 1
        
    seq.asc(length_mus=0.5)  # 2nd step
#     seq.asc(length_mus = 10, green=False, orange = True) #green on, AOM off

    seq.asc(timetagger=False,green=True,orange=False, length_smpl=AOMdelay_sample)
    seq.asc(timetagger=True,green=True,orange=True, length_smpl=length_laserreadout_smpl)    
    seq.asc(timetagger=True,green=False,orange=True, length_smpl=AOMdelay_sample)

    seq.asc(length_mus= length_lasersafety_ns / 1e3)
    return seq





def awg_AddWait(seq, time, orange=False, green=False):
    enable_reuse_segments = False
    if time < 0.:
        for i in range(3): 
            print ('Negative time for sequence in AWG!!!')
    elif time > 10.:
        if int(time / (32 / 12.)) >= 1:
            seq.start_new_segment('wait_loop', reuse_segment=enable_reuse_segments,
                                  loop_count=int(time / (32 / 12.)))
            seq.asc(length_smpl=32000, orange=orange, green=green)
        seq.start_new_segment('wait_additional')
        seq.asc(length_mus=time % (32 / 12.), orange=orange, green=green)
    else:
        seq.asc(length_mus=time, orange=orange, green=green)
    return seq

def AddRfSafety(seq):
    rf_safety = 1
    seq.asc(length_mus=rf_safety)
    return seq

# taus = np.linspace(100,1000,91)
# taus = np.arange(0.01,130,1.)
taus = np.arange(1., 10, .2)


seqtest = awg_readout_test(taus)
seqtest.awg = awg




# pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
#                                            controlled_variable= taus,
#                                            number_of_lasers=2*len(taus), 
#                                            laser_ignore_list=[], 
#                                            alternating=True, 
#                                            units=('s', 'arb. u.'))




In [26]:

pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
                                           controlled_variable= [1],
                                           number_of_lasers=1, 
                                           laser_ignore_list=[], 
                                           alternating=False, 
                                           units=('s', 'arb. u.'))

In [27]:
seqtest.write_seq()

writing sequence wait on AWG_M8190A_qudi ch 1 took 0.0140 seconds
writing sequence wait on AWG_M8190A_qudi ch 2 took 0.0100 seconds
MCAS 'hahn_echo' has been generated.


In [28]:
seqtest.run_sequence()

writing sequence hahn_echo on AWG_M8190A_qudi ch 1 took 1.5476 seconds
writing sequence hahn_echo on AWG_M8190A_qudi ch 2 took 1.5324 seconds


In [ ]:
seqtest.stop_awgs()